In [1]:
import pandas as pd
import os

In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,16 mins 55 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,27 days
H2O cluster name:,H2O_from_python_Р”РёРјР°_Рё_РќР°СЃС‚СЏ_5c8o19
H2O cluster total nodes:,1
H2O cluster free memory:,3.229 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [3]:
X = pd.read_csv('X.csv', index_col = 0)

C:\conda\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
y = pd.read_csv('y.csv', index_col = 0, header = None, names = ['id', 'y'])

In [5]:
meter = 0
df_meter = X[X.meter == meter]
buildings = df_meter.building_id.unique()
building = buildings[0]
df_prep = df_meter[df_meter.building_id == building]
X_train = df_prep.drop(['meter', 'building_id'], axis=1)
y_train = y.loc[X_train.index]


In [6]:
df = pd.concat([X_train, y_train], axis = 1)

split = int((len(df)*0.9)//1)
df_train = df.iloc[:split]
df_test = df.iloc[split:]

hdf_train = h2o.H2OFrame(df_train)
hdf_test = h2o.H2OFrame(df_test)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
df_train.head(1)

,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,had_air_temperature,had_cloud_coverage,had_dew_temperature,had_precip_depth_1_hr,had_sea_level_pressure,had_wind_direction,had_wind_speed,tm_day_of_week,tm_hour_of_day,y
103,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,1,0,1,0,1,1,1,0,0,3.190624


In [8]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "ashrae")

In [9]:
aml.train(training_frame = hdf_train,leaderboard_frame = hdf_test, x = list(X_train.columns), y = 'y')

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [10]:
aml.leaderboard.head() #0.106653 - 5s, 10s - 0.0863501, 30s - 0.0861425, 60s - 0.0848381, 600s 0.082

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_5_AutoML_20191201_143243,0.0820335,0.286415,0.0820335,0.212338,0.054655
StackedEnsemble_AllModels_AutoML_20191201_143243,0.08216,0.286636,0.08216,0.212492,0.0546426
StackedEnsemble_BestOfFamily_AutoML_20191201_143243,0.0828114,0.28777,0.0828114,0.216896,0.0548643
GBM_3_AutoML_20191201_143243,0.0850488,0.291631,0.0850488,0.211387,0.0553869
GBM_4_AutoML_20191201_143243,0.0853481,0.292144,0.0853481,0.214694,0.0555155
DRF_1_AutoML_20191201_143243,0.0861425,0.2935,0.0861425,0.220789,0.0556687
GBM_1_AutoML_20191201_143243,0.0885728,0.297612,0.0885728,0.215585,0.0565061
GBM_2_AutoML_20191201_143243,0.0893495,0.298914,0.0893495,0.213875,0.0566759
GLM_grid_1_AutoML_20191201_143243_model_1,0.134962,0.367372,0.134962,0.303001,0.0685641


In [11]:
h2o.estimators.xgboost.H2OXGBoostEstimator.available()

Cannot build an XGBoost model - no backend found.


False

In [12]:
directory = os.path.join(str(meter), str(building))
if not os.path.exists(directory):
    os.makedirs(directory)

In [13]:
h2o.save_model(model=aml.leader, path=directory, force=True)

'C:\\Users\\Р”РёРјР° Рё РќР°СЃС‚СЏ\\Documents\\kaggle\\0\\105\\GBM_5_AutoML_20191201_143243'

In [6]:
for meter in [0, 1, 2, 3]:
    df_meter = X[X.meter == meter]
    buildings = df_meter.building_id.unique()
    error_list = []
    for building in buildings:
        try:
            directory = os.path.join(str(meter), str(building))
            if os.path.exists(directory):
                continue
            df_prep = df_meter[df_meter.building_id == building]
            X_train = df_prep.drop(['meter', 'building_id'], axis=1)
            y_train = y.loc[X_train.index]

            df = pd.concat([X_train, y_train], axis = 1)

            split = int((len(df)*0.9)//1)
            df_train = df.iloc[:split]
            df_test = df.iloc[split:]

            hdf_train = h2o.H2OFrame(df_train)
            hdf_test = h2o.H2OFrame(df_test)

            aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "ashrae")
            aml.train(training_frame = hdf_train,leaderboard_frame = hdf_test, x = list(X_train.columns), y = 'y')
            print(aml.leaderboard.head())
            
            if not os.path.exists(directory):
                os.makedirs(directory)
            h2o.save_model(model=aml.leader, path=directory, force=True)
            print(str(meter)+ '_' + str(building))
        except Exception as ex:
            print('ERROR' + str(meter)+ '_' + str(building))
            print(ex)
            error_list.append(str(meter)+ '_' + str(building))
            